在这个教程中，你将会学到怎么用pydeck制作可交互地图可视化  

提供的基础数据是：
<div class="alert alert-info"><h2>提供的基础数据是：</h2><p>   
    数据：<br>  
    1.出租车GPS集计栅格OD<br>  

 </p></div>

# 什么是deck.gl

deck.gl 是一个WebGL支持的框架，用于大型数据集的可视化探索性数据分析。  
deck.gl是由uber开发并开源出来的基于WebGL的大数据量可视化框架。它具有提供不同类型可视化图层，GPU渲染的高性能，React和Mapbox GL集成，结合地理信息数据（GPS）的特点。  
[deck.gl的官方网站](https://deck.gl)  
[deck.gl的官方example](https://deck.gl/#/examples/overview)

deck.gl提供了python接口，非常方便  
[pydeck的官方document](https://pydeck.gl/)

# 安装

安装pydeck
>pip install pydeck

安装jupyter notebook的pydeck插件  
>jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck  
jupyter nbextension enable --sys-prefix --py pydeck

deck.gl的地图地图是基于mapbox的，因此各位得申请一个mapbox的apikey，在这个链接  
[access token](https://docs.mapbox.com/help/how-mapbox-works/access-tokens/)

# 读取数据

In [ ]:
mapbox_key = '你的mapboxkey'

In [83]:
#导入必要的包
import pandas as pd
import numpy as np

#导入pydeck
import pydeck as pdk

In [2]:
#读取OD数据
TaxiOD = pd.read_csv(r'data-sample/TaxiOD.csv')
TaxiOD.columns = ['VehicleNum', 'Stime', 'SLng', 'SLat', 
       'ELng', 'ELat','Etime']
TaxiOD.head(5)

,VehicleNum,Stime,SLng,SLat,ELng,ELat,Etime
0,22223,00:03:23,114.167465,22.562468,114.225235,22.552750,00:10:48
1,22223,00:11:33,114.227150,22.554167,114.229218,22.560217,00:15:19
2,22223,00:17:13,114.231354,22.562166,114.255798,22.590967,00:29:06
3,22223,00:36:45,114.240196,22.563650,114.119965,22.566668,00:54:42
4,22223,01:01:14,114.135414,22.575933,114.166748,22.608267,01:08:17


In [84]:
len(TaxiOD)

464718

如果我们直接可视化46万条数据，pydeck还是一下子没法处理那么多，亲测几万条数据还是可以的，但是上十万就会卡死不显示。  
所以这里还是对数据的经纬度保留三位小数集计后再进行可视化

In [47]:
TaxiOD['SLng'] = TaxiOD['SLng'].round(3)
TaxiOD['SLat'] = TaxiOD['SLat'].round(3)

In [48]:
Taxi_O_agg = TaxiOD.groupby(['SLng', 'SLat'])['VehicleNum'].count().reset_index()
Taxi_O_agg = Taxi_O_agg[(100<Taxi_O_agg['SLng']) & (Taxi_O_agg['SLng']<180)]

# 可视化

## Hexagon

[deckgl官方document的HexagonLayer](https://deck.gl/#/documentation/deckgl-api-reference/layers/hexagon-layer?section=coloraggregation-string-optional-)

In [ ]:
import pydeck 
# Define a layer to display on a map
layer = pydeck.Layer(
     'HexagonLayer',
    Taxi_O_agg,                                   #数据在此输入
    get_position=['SLng', 'SLat'],                #指定经纬度的列
    getColorWeight = 'VehicleNum',                #指定集计数量的列
    getElevationWeight = 'VehicleNum',            #指定集计数量的列
     auto_highlight=True,
     elevation_scale=10,
     pickable=True,
     radius = 200,                                #Hexagon的大小
     elevation_range=[0, 500],                    #Hexagon的高低
     extruded=True,
     coverage=1)                                  #Hexagon覆盖比例

# Set the viewport location
view_state = pdk.ViewState(
    longitude=114.027465,                         #地图底图中心经纬度
    latitude=22.632468,                           #地图底图中心经纬度
    zoom=10,                                      #地图底图缩放等级
    min_zoom=5,
    max_zoom=12,
    pitch=40,                                     #地图底图俯视角度
    bearing=-10)                                  #地图底图选转角度

# Render
r = pdk.Deck(layers=[layer],                       #创建deckgl实例
             initial_view_state=view_state,
             mapbox_key = mapbox_key,               #你的mapbox_key
             map_style =  'mapbox://styles/mapbox/dark-v9'#mapbox底图的风格
            )
r.show()

<img src="https://gitee.com/ni1o1/pygeo-tutorial/raw/master/resource/deckHexagonLayer.png" style="width:1000px">

## Heatmap

In [ ]:
import pydeck 
# Define a layer to display on a map
layer = pydeck.Layer(
     'HeatmapLayer',
    Taxi_O_agg,                                   #数据在此输入
    get_position=['SLng', 'SLat'],                #指定经纬度的列
    getWeight = 'VehicleNum',                     #指定权重
     )

# Set the viewport location
view_state = pdk.ViewState(
    longitude=114.027465,
    latitude=22.632468,
    zoom=10,
    min_zoom=5,
    max_zoom=12,
    pitch=40,
    bearing=-10)

# Render
r = pdk.Deck(layers=[layer], 
             initial_view_state=view_state,
            mapbox_key = mapbox_key,
              map_style =  'mapbox://styles/mapbox/dark-v9'
            )
r.show()

<img src="https://gitee.com/ni1o1/pygeo-tutorial/raw/master/resource/deckheatmap.png" style="width:1000px">